# Lecture 15: Long-range interactions

## Physics 7810, Spring 2020

## 15.1 - Overview

For systems of particles with short-range interactions, the potential energy, forces, and other properties can be calculated in a straightforward and efficient way using the methods described previously, using the minimum-image convention (for periodic systems) or spherical truncation of interactions (in general). Approximate long-range corrections to the potential energy and pressure may also be applied.

The situation is more complicated in systems with *long-range* interactions that fall off with distance $r$ no faster than $\sim r^{-d}$, where $d$ is the spatial dimensionality. Important examples include systems of particles with Coulombic, dipolar, or gravitational interactions. In such systems, simple truncation of pair interactions leads to highly inaccurate results, and calculating all pairwise interactions without truncation in a brute force fashion entails $O(N^2)$ computational cost in a system of $N$ particles, and yields ill-defined properties in systems with periodic boundary conditions.

A variety of methods have been developed to deal with these issues, including the *Ewald summation method* for computing electrostatic interactions in periodic systems of charged particles, in which a conditionally convergent sum over a periodic array of unit cells is replaced by two rapidly convergent sums, one in real space and the other in Fourier space. When optimized with respect to a free convergence parameter, the computational cost of the Ewald summation method scales as $O(N^{3/2})$.

For large $N$, even better scaling can be achieved with *grid-based methods* such as the smooth particle mesh Ewald (SPME) and particle-particle particle-mesh (PPPM) methods, in which the real-space charge density is mapped onto a regular grid, enabling use of the fast Fourier transform (FFT) to compute the Fourier-space part of the Ewald sum. The computational complexity of these methods is that of the FFT algorithm, $O(N \ln N)$.

Finally, multiscale methods such as the *fast multipole method* (FMM) achieve $O(N)$ scaling, albeit at the cost of additional algorithmic complexity, and are the fastest methods for simulating very large systems. Unlike Ewald-based methods, the FFM method is easily adapted to non-periodic systems, which makes it an attractive method for cosmological and fluid dynamics simulations (but the PPPM method is also an attractive option).

## 15.2 - Periodic boundary conditions and long-range interactions

Periodic boundary conditions are conventionally employed to minimize finite-size effects in computer simulations, but may introduce complications in the case of long-range interactions. Assuming a pairwise additive potential, the potential energy for a 3D system can be written as a lattice sum over all periodic images of the simulation box,

$$
U = \frac{1}{2} {\sum_{\bf n}}^\prime \sum_{i = 1}^N \sum_{j = 1}^N u(|{\bf r}_{ij} + {\bf n} L|).
$$

Here the sum on ${\bf n} = (n_x, n_y, n_z)$ ranges over all triplets of integers, and $L$ is the linear dimension of the periodic box, assumed to be cubic. The prime on the sum over ${\bf n}$ indicates that we omit the self-interaction term $i = j$ for ${\bf n} = 0$.

Previously, we avoided summing over all periodic cells by truncating the pair potential at a cutoff distance $r_c < L/2$, in which case the total potential energy for a spherically symmetric pair potential can be written

$$
U = \sum_{i <j}^N u(r_{ij}) + \frac{N \rho}{2} \int_{r_c}^\infty dr\ u(r)\ 4 \pi r^2.
$$

Note that for power-law potentials of the form $u(r) \propto r^{-n}$, the tail 'correction' diverges for $n \leq 3$, and the simple truncation scheme fails. In such cases, special care must be taken in evaluating long-range interactions, and the lattice sum described on the previous slide needs to be evaluated explicitly.

Here we discuss the Ewald summation method for evaluating such lattice sums for point charges ($n = 1$) and point dipoles ($n = 3$).

## 15.3 - Ewald sums for point charges

The potential energy of a periodic system of point charges can be written

$$
U^\mathrm{coul} = \frac{1}{2} {\sum_{\bf n}}^\prime \sum_{i = 1}^N \sum_{j = 1}^N \frac{q_i q_j}{|{\bf r}_{ij} + {\bf n} L|},
$$

where, as before, the sum on ${\bf n} = (n_x, n_y, n_z)$ ranges over all triplets of integers, and $L$ is the linear dimension of the periodic box, assumed to be cubic. The prime on the sum over ${\bf n}$ indicates that we omit the self-interaction term $i = j$ for ${\bf n} = 0$. We assume that the system is electrically neutral, $\sum_{i = 1}^N q_i = 0$. To simplify the notation, we use Gaussian units (hence avoiding factors of $4 \pi \epsilon_0$). 

This sum is *conditionally convergent*: the value of the sum depends on the order in which the terms in the sum over ${\bf n}$ are added up.

To proceed further, it's convenient to write the total energy as

$$
U^\mathrm{coul} = \frac{1}{2} \sum_{i = 1}^N q_i \Phi({\bf r}_i),
$$

where $\Phi({\bf r}_i)$ is the electrostatic potential at the position of ion $i$,

$$
\Phi({\bf r}_i) = {\sum_{\bf n}}^\prime \sum_{j = 1}^N \frac{q_j}{|{\bf r}_{ij} + {\bf n} L|}.
$$

This conditionally convergent sum can be converted into two absolutely convergent sums by introducing a convergence factor, that cleverly exploits the linearity of Poisson's equation, $\nabla^2 \Phi({\bf r}) = - 4 \pi \rho({\bf r})$, where $\rho({\bf r})$ is the charge density. For point charges, the charge density is a sum of delta functions,

$$
\rho({\bf r}) = \sum_{i = 1}^N q_i \delta({\bf r} - {\bf r}_i).
$$

The Ewald method is a clever trick to exploit the superposition principle to recast the potential energy into an absolutely convergent sum.

The first step is to consider surrounding every charge $q_i$ with a diffuse charge distribution of equal and opposite total charge, thus screening the point charge and ensuring that the potential produced by each charge goes to zero rapidly at large distances.

The potential energy contributions from these screened charges can be summed directly, as they are short-ranged, but we then need to compensate for the addition of fictitious screening charge distributions.

The compensating charge distribution is just the screening charge distribution with opposite sign, a mirrored version of the diffuse charge distribution that was added to the point charges.

This compensating charge distribution is a smoothly varying periodic function that gives rise to potential energy contributions that converge rapidly in Fourier space.

Point charges can be represented by a superposition of screened charges with their compensating distributions. Superposition ensures that the electrostatic potential produced by the point charge distribution shown on the left is equal to the sum of the potentials produced by the two charge distributions shown on the right. 

<img src="images/Frenkel_Fig_12.1.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

A conventional and convenient choice of screening charge distribution is a Gaussian distribution,

$$
\rho_G({\bf r}) = - q_i (\alpha^2/\pi)^{3/2} \exp(- \alpha^2 r^2).
$$

As we'll see, this is particularly convenient because the Fourier transform of a Gaussian is a Gaussian. The parameter $\alpha$ is chosen for computational efficiency, and can be used to validate the Ewald procedure, as the total energy can't depend on the value of $\alpha$.

The charge density of a point charge $q_i$ surrounded by a Gaussian charge distribution with net charge $-q_i$ located at the origin is

$$
\rho({\bf r}) = q_i \left[ \delta({\bf r}) - (\alpha^2/\pi)^{3/2} \exp(- \alpha^2 r^2) \right]
$$

**Fourier-space contribution**

Given the Gaussian form above, a periodic sum of compensating charge distributions has the form

$$
\rho_1({\bf r}) = \sum_{j = 1}^N \sum_{\bf n} q_j (\alpha^2/\pi)^{3/2} \exp \left[ - \alpha^2 |{\bf r} - ({\bf r}_j + {\bf n} L)|^2  \right].
$$

To compute the electrostatic potential due to this charge distribution we use Poisson's equation in Fourier space,

$$
k^2 \Phi_1({\bf k}) = 4 \pi \rho_1({\bf k}).
$$


Here

$$
\rho_1({\bf k}) = \int_V d{\bf r} e^{- i {\bf k} \cdot {\bf r}} \rho_1 ({\bf r})
= \int_V d{\bf r} e^{- i {\bf k} \cdot {\bf r}} \sum_{j = 1}^N \sum_{\bf n} q_j (\alpha^2/\pi)^{3/2} \exp \left[ - \alpha^2 |{\bf r} - ({\bf r}_j + {\bf n} L)|^2  \right]
$$
$$
= \int_\mathrm{all\ space} d{\bf r} e^{- i {\bf k} \cdot {\bf r}} \sum_{j = 1}^N q_j (\alpha^2/\pi)^{3/2} \exp \left[ - \alpha^2 |{\bf r} - {\bf r}_j|^2  \right]
$$
$$
= \sum_{j = 1}^N q_j e^{- i {\bf k} \cdot {\bf r}_j} \exp(-k^2 / 4 \alpha^2).
$$

Inserting this into Poisson's equation then gives

$$
\Phi_1({\bf k}) = \frac{4 \pi}{k^2} \sum_{j = 1}^N q_j e^{- i {\bf k} \cdot {\bf r}_j} \exp(-k^2 / 4 \alpha^2).
$$

This expression is valid for $k \neq 0$, which is related to the sum being conditionally convergent. The real-space potential is obtain from an inverse Fourier transform,

$$
\phi_1({\bf r}) = \frac{1}{V} \sum_{{\bf k} \neq 0} \Phi_1({\bf k}) e^{i {\bf k} \cdot {\bf r}_j}
= \frac{4 \pi q_j}{k^2 V} \sum_{j = 1}^N \sum_{{\bf k} \neq 0} e^{i {\bf k} \cdot ({\bf r} - {\bf r}_j)} \exp(-k^2 / 4 \alpha^2),
$$

so the long-range contribution to the energy is

$$
U_\mathrm{long-range}^\mathrm{coul} = \frac{1}{2} \sum_{i = 1}^N q_i  \phi_1({\bf r}_i)
= \frac{1}{2} \sum_{{\bf k} \neq 0} \sum_{i, j = 1}^N \frac{4 \pi q_i q_j}{k^2 V} e^{i {\bf k} \cdot ({\bf r}_i - {\bf r}_j)} \exp(-k^2 / 4 \alpha^2)
$$

$$
= \frac{1}{2 V} \sum_{{\bf k} \neq 0} \frac{4 \pi}{k^2} |\rho({\bf k})|^2 \exp(-k^2 / 4 \alpha^2)
$$

where $\rho({\bf k}) = \sum_{i = 1}^N q_i \exp(i {\bf k} \cdot {\bf r}_i)$ is the Fourier transform of the charge density.



**Self-interaction correction**

The Fourier-space contribution to the potential energy includes spurious self-interaction terms $(1/2) q_i \Phi_\mathrm{gauss}({\bf r}_i)$ due to the interaction between a Gaussian charge cloud of charge $q_i$ and a point charge $q_i$ located at the center of the Gaussian. These terms need to be subtracted out, so we need to calculate the corresponding potential energy, which involved computing the electrostatic potential energy at the center of a Gaussian cloud of charge,

$$
\rho_\mathrm{gauss}({\bf r}) = q_i (\alpha^2/\pi)^{3/2} \exp(- \alpha^2 r^2).
$$

We start with Poisson's equation in spherical coordinates, making use of the spherical symmetry of the charge distribution to obtain

$$
- \frac{1}{r} \frac{\partial^2}{\partial r^2} \left[ r \Phi_\mathrm{gauss}(r) \right] = 4 \pi \rho_\mathrm{gauss}(r)
$$
or
$$
- \frac{\partial^2}{\partial r^2} \left[ r \Phi_\mathrm{gauss}(r) \right] = 4 \pi r \rho_\mathrm{gauss}(r)
$$

This can be integrated to obtain

$$
- \frac{\partial}{\partial r} \left[ r \Phi_\mathrm{gauss}(r) \right]
= \int_\infty^r dr^\prime 4 \pi r \rho_\mathrm{gauss}(r^\prime)
= - 2 \pi q_i (\alpha^2 / \pi)^{3/2} \int_r^\infty {dr^\prime}^2 \exp(- \alpha^2 {r^\prime}^2)
$$
$$
= - 2 q_i (\alpha^2 / \pi)^{1/2} \exp(- \alpha^2 r^2),
$$

and a second integration yields

$$
r \Phi_\mathrm{gauss}(r) = 2 q_i (\alpha^2 / \pi)^{1/2} \int_0^r dr^\prime \exp(- \alpha^2 {r^\prime}^2)
= q_i \mathrm{erf}(\alpha r)
$$
or
$$
\Phi_\mathrm{gauss}(r) = q_i \frac{\mathrm{erf}(\alpha r)}{r},
$$

where the *error function* is defined as

$$
\mathrm{erf}(x) \equiv \frac{2}{\sqrt{\pi}} \int_0^x \exp(-t^2) dt
$$

To calculate the self-energy correction, we need to calculate $\Phi_\mathrm{gauss}(r)$ at $r = 0$, which is

$$
\Phi_\mathrm{gauss}(0) = \frac{2 q_i \alpha}{\sqrt{\pi}},
$$

so the total self-interaction contribution is

$$
U_\mathrm{self}^\mathrm{coul} = \frac{1}{2} \sum_{i = 1}^N q_i \Phi_\mathrm{gauss}(0) 
= \frac{\alpha}{\sqrt{\pi}} \sum_{i = 1}^N q_i^2.
$$

This spurious contribution needs to be *subtracted* from the Fourier space energy.

**Real-space contributions**

To compute the short-range part of the Ewald sum, we need to compute interactions among Gaussian-screened point charges. This can be done easily using the result from the last section for the potential due to a Gaussian charge distribution, and we immediately obtain

$$
\Phi_\mathrm{short-range}({\bf r}) = \frac{q_i}{r} - \frac{q_i}{r} \mathrm{erf}(\alpha r)
= \frac{q_i}{r} \mathrm{erfc}(\alpha r)
$$

where $\mathrm{erfc}$ is the complementary error function,

$$
\mathrm{erfc}(x) \equiv \frac{2}{\sqrt{\pi}} \int_x^\infty \exp(-t^2) dt.
$$

The total short-range contribution to the Coulomb potential energy is therefore

$$
U_\mathrm{short-range}^\mathrm{coul} = \frac{1}{2} \sum_{i \neq j}^N q_i q_j \frac{\mathrm{erfc}(\alpha r_{ij})}{r_{ij}}
$$

**Total Coulomb energy**

Putting all the pieces together, the total electrostatic energy becomes

$$
U^\mathrm{coul} = \frac{1}{2} \sum_{i \neq j}^N q_i q_j \frac{\mathrm{erfc}(\alpha r_{ij})}{r_{ij}} + \frac{1}{2V} \sum_{{\bf k} \neq 0} \frac{4 \pi}{k^2} |\rho({\bf k})|^2 \exp(- k^2/4 \alpha^2) - \frac{\alpha}{\sqrt{\pi}} \sum_{i = 1}^N q_i^2.
$$

## 15.4 - The particle-particle particle-mesh method

We've seen that the long-range part of the potential energy in the Ewald summation method involves computing a Fourier transform of the charge density,

$$
\rho({\bf k}) = \int d{\bf r} \rho({\bf r})\ e^{-i {\bf k} \cdot {\bf r}} = \sum_{i = 1}^N q_i e^{-i {\bf k} \cdot {\bf r}_i}.
$$

Given this, it's natural to imagine that this calculation could be speeded up considerably through the use of the fast Fourier transform (FFT), and in fact such an FFT-based method, the *particle-particle particle-mesh* (PPPM) algorithm, was developed by Hockney and Eastwood in the 1980s. Two other mesh-based methods, the *particle-mesh Ewald* (PME) and *smooth particle-mesh Ewald* (SPME) algorithms, were introduced in the early 1990s, but as discussed in detail by Deserno and Holm (*J. Chem. Phys.* **109**, 7678 (1998)), all of these methods are closely related, and a optimized PPPM algorithm turns out to be advantageous from the standpoint of flexibility and accuracy. For these reasons, we'll only describe the PPPM method here.

The source of sacred wisdom about PPPM.

<img src="images/computer simulation using particles.png" alt="Drawing" style="width: 400px;">


As with the Ewald method, the P$^3$M method relies on a splitting of the total Coulomb potential into two parts. The  short-range part (the 'particle-particle' part) can be efficiently evaluated in $O(N)$ time in the usual way, using cell-search methods, while the long range ('particle-mesh') part is computed using FFTs, in $O(N \ln N)$ time, so the overall method scales as $O(N \ln N)$.

The basic idea behind the PPPM method is the same as the Ewald method, that is to split the Coulomb potential into two parts using the trivial identity

$$
\frac{1}{r} = \frac{f(r)}{r} + \frac{1-f(r)}{r}.
$$

Recall that, for the Ewald method discussed above, $f(r) = \mathrm{erfc}(\alpha r)$. In fact, if one uses the same Gaussian screening function as in the Ewald summation, the PPPM becomes very similar to the Ewald method. However, this not the only choice, nor is it necessarily the optimal one.

Recall the Fourier-space contribution to the energy in the Ewald method,

$$
U_\mathrm{long-range}^\mathrm{coul} = \frac{1}{2 V} \sum_{{\bf k} \neq 0} \frac{4 \pi}{k^2} |\rho({\bf k})|^2 \exp(-k^2 / 4 \alpha^2).
$$

Following Deserno and Holm, this can be written as

$$
U_\mathrm{long-range}^\mathrm{coul} = \frac{1}{2} \sum_{i = 1}^N q_i \left( 
\frac{1}{V} \sum_{{\bf k} \neq 0} g({\bf k}) \gamma ({\bf k}) \rho ({\bf k}) e^{i {\bf k} \cdot {\bf r}_i}
\right)
$$

$$
= \frac{1}{2} \sum_{i = 1}^N q_i \phi^k({\bf r}_i).
$$

Here, $g({\bf k}) = 4 \pi / k^2$ is the Fourier transform of the Coulomb Green's function $1/r$, and $\gamma ({\bf k}) = \exp(-k^2/4 \alpha^2)$ is the Fourier transform of the Gaussian screening function.

Here we've defined

$$
\phi^k({\bf r}_i) = \frac{1}{V} \sum_{{\bf k} \neq 0} g({\bf k}) \gamma ({\bf k}) \rho ({\bf k}) e^{i {\bf k} \cdot {\bf r}_i},
$$

which is the Fourier-space contribution to the potential at the location of charge $i$. From this definition of $\phi^k({\bf r}_i)$, it can be seen that its Fourier transform is

$$
\phi^k({\bf k}) = g({\bf k}) \gamma ({\bf k}) \rho ({\bf k}).
$$

But products in reciprocal space correspond to convolutions in real space, so this tells us that the reciprocal space contribution to the electrostatic potential is the original charge distribution convoluted with a 'smearing function' $\gamma({\bf r})$. For the standard Ewald sum $\gamma({\bf r})$ is a Gaussian, which corresponds to choosing the splitting function $f(r)$ to be a complementary error function, but it should be clear that there are many other choices.

In fact, these two points of view are equivalent: we can split either the *charge density* or the *electrostatic potential*.

Evaluation of the above expression using a discrete FFT involves the following steps:

1. *Charge assignment*. Charges are assigned (interpolated) to grid points.

2. *Solve Poisson's equation*. The Poisson equation for a discrete charge distribution is solved using FFTs.

3. *Force assignment* (for MD simulations). Once the electrostatic energy has been obtained by solving the Poisson equation, the forces are calculated and assigned back to the particles.

At each step in this procedure, several distinct approaches are possible. The various options have been carefully analyzed by Deserno and Holm (*J. Chem. Phys.* **109**, 7678 (1998)), and their conclusions are summarized in the following discussion, where we consider each step in detail.

We start by defining a finely-spaced grid in the simulation box. For simplicity, we assume a cubic grid with spacing $\ell$, where the number of grid points along each dimension of the simulation box is $n_g = L / \ell$, where $n_g$ is often chosen to be a power of 2 (but need not be). The grid charge density $\rho_g({\bf r}_g)$ is defined at each grid point ${\bf r}_g = \ell (n_x, n_y, n_z)$, where $n_\alpha = 0, ..., n_g - 1$.

To assign charges to a grid, a charge assignment function, $W({\bf r})$, is introduced. In one dimension, for example, the fraction of a unit charge at position $x$ assigned to a grid (mesh) point at position $x_g$ is $W(x_g - x)$. Given a 3D charge distribution $\rho({\bf r})= \sum_{i = 1}^N q_i \delta ({\bf r} - {\bf r}_i)$, the *grid charge density* $\rho_g({\bf r})$ is defined as

$$
\rho_g({\bf r}_g) = \frac{1}{\ell^3} \int_V d{\bf r} W({\bf r}_g - {\bf r}) \rho({\bf r})
= \frac{1}{\ell^3} \sum_{i = 1}^N q_i W({\bf r}_g - {\bf r}_i).
$$

The charge assignment function $W$ must be chosen to ensure conservation of charge, i.e., the sum of fractional charges assigned to grid points must equal the original charge $q_i$.

The assignment function $W$ should also have as small a number of 'supporting' grid points as possible, to minimize computational cost, and it should be smooth and easy to calculate.

Hockney and Eastwood introduced a family of $P$th-order assignment functions, which distribute a given charge to the neighboring $P$ grid points along each dimension. These assignment functions are most easily defined in Fourier space, as

$$
W^{{P}}(k_x) = \ell \left( \frac{\sin(k_x \ell / 2)}{k_x \ell / 2} \right)^P.
$$

The quantity in parentheses is the Fourier transform of the boxcar function,

$$
w(x) = \left\{
\begin{array}{ll}
0, & x < -\ell/2 \\
1/\ell, & -\ell/2 \leq x < \ell/2 \\
0, & x \geq \ell/2
\end{array}
\right.,
$$

so in real space, $W^{(P)}(x)$ is a convolution of a series of $P$ boxcar functions. The lowest-order cases, $P = 1, 2, 3$ are called 'nearest grid point', 'cloud in cell', and 'triangular-shaped cloud', respectively. 

The functional forms for $W^{(P)}(x)$ up to $P = 7$ are tabulated in Appendix E of Deserno and Holm (*J. Chem. Phys.* **109**, 7678 (1998)). Here we consider the function that distributes charges across the three nearest grid points ($P = 3$, 'triangular-shaped cloud'),

$$
W^{(3)}(x) = \left\{
\begin{array}{ll}
\frac{3}{4} - \left( \frac{x}{\ell} \right)^2, & |x| \leq \frac{1}{2} \ell \\
\frac{1}{2} \left(\frac{3}{2} - \frac{|x|}{\ell} \right)^2, & \frac{1}{2} \ell \leq |x| \leq \frac{3}{2} \ell \\
0, & \mathrm{otherwise}
\end{array}
\right.,
$$

where $x$ is the distance between the charge and any grid point. Suppose that the nearest grid point is $n$, and the distance from it to the charge, in units of $\ell$, is $x^\prime = (x - x_n) / \ell$. Then the three nonzero weights at grid points $n, n \pm 1$ can be written in terms of $x^\prime$ as

$$
W^{(3)}(x^\prime) = \left\{
\begin{array}{ll}
\frac{1}{2}\left( \frac{1}{2} + x^\prime \right)^2, & \mathrm{at}\ \ n+1 \\
\frac{3}{4} - {x^\prime}^2, & \mathrm{at}\ \ n \\
\frac{1}{2}\left( \frac{1}{2} - x^\prime \right)^2, & \mathrm{at}\ \ n-1
\end{array}
\right.
$$

Assignment of a charge to its three nearest grid points in one dimension, using the triangular-shaped cloud weighting function. The vertical dashed lines are the cell boundaries and the vertical solid lines are the grid points at the center of each cell.

<img src="images/A&T_Fig_6.3.png" alt="Drawing" style="width: 800px;">

Figure from *Computer Simulation of Liquids*, by Mike Allen & Dominic Tildesley.

In 3D, the 27 nearest grid points have a weight given by

$$
W^{(3)}({\bf r}_g) = W^{(3)}(x^\prime) W^{(3)}(y^\prime) W^{(3)}(z^\prime).
$$

Once the charge distribution has been assigned to the grid, its discrete Fourier transform is

$$
\rho_g({\bf k}) = \ell^3 \sum_{n_x = 0}^{n_g - 1} \sum_{n_y = 0}^{n_g - 1} \sum_{n_z = 0}^{n_g - 1}
\rho_g({\bf r}_s) e^{- i {\bf k} \cdot {\bf r}_g}
= \ell^3 \sum_{{\bf r}_g} \rho_g({\bf r}_s) e^{- i {\bf k} \cdot {\bf r}_g},
$$

where $\rho_g({\bf k})$ is defined for the finite set of wavevectors ${\bf k} = (2 \pi / L) (m_x, m_y, m_z)$, with
$|m_\alpha| \leq (n_g - 1)/2$. This is in a form suitable for evaluation using the FFT. The general procedure is to store $\rho_g({\bf r}_s)$ as an array of complex numbers, with the imaginary parts set to zero, then use a 3D complex-to-complex FFT to obtain $\rho_g({\bf k})$. I highly recommend using FFTW (pretty much the universal choice these days). A good introduction to discrete Fourier transforms and the FFT method can be found in *Numerical Recipes* (Press et al.).

Once $\rho_g({\bf k})$ has been calculated, the long-range part of the electrostatic energy is given by

$$
U_\mathrm{long-range}^\mathrm{coul} = \frac{1}{2 V} \sum_{{\bf k} \neq 0} G({\bf k}) |\rho_g({\bf k})|^2.
$$

Here $G({\bf k})$ is the *influence function*, which in the conventional Ewald method has the form $G({\bf k}) = (4 \pi /k^2) \exp(- \alpha^2 k^2)$, but for mesh-based methods $G({\bf k})$ needs to be optimized to correct for the distortion of the isotropic charge distribution due to interpolation onto a cubic grid, so in general we have

$$
U_\mathrm{long-range}^\mathrm{coul} = \frac{1}{2 V} \sum_{{\bf k} \neq 0} G_\mathrm{opt}({\bf k}) |\rho_g({\bf k})|^2,
$$

where $G_\mathrm{opt}({\bf k})$ is the optimized influence function, designed to minimize errors in the computed forces (or energies) relative to the Ewald method (the gold standard). Expressions for $G_\mathrm{opt}({\bf k})$ can be found in Allen & Tildesley, Hockney and Eastwood, or Deserno and Holm (omitted here because they're rather complicated).

$U_\mathrm{long-range}^\mathrm{coul}$ can also be calculated in real space, in terms of the grid positions ${\bf r}$,

$$
U_\mathrm{long-range}^\mathrm{coul} = \frac{1}{2} \ell^3 \sum_{{\bf r}_g} \rho_g({\bf r}_g)
\left( \rho_g \star G_\mathrm{opt} \right)({\bf r}_g),
$$

where the convolution is $\rho_g \star G_\mathrm{opt} = {\cal F}^{-1} [\rho_g({\bf k}) G_\mathrm{opt}({\bf k})]$, where ${\cal F}^{-1}$ denotes an inverse discrete Fourier transform.

The real-space electrostatic potential and force can be obtained in a similar way, via inverse FFTs.

The force on a particle, i.e., the derivative of the potential, can be calculated in three different ways:

1. Using a finite-difference scheme in real space applied to the potential at neighboring grid points.

2. Analytically differentiating the real space potential energy on the previous slide, which requires the gradient of the assignment function.

3. Differentiating in Fourier space to obtain the electric field by multiplying the potential by $i{\bf k}$ and taking a inverse FFT.

The relative performance and accuracy of these three methods have been assessed by Deserno and Holm (*J. Chem. Phys.* **109**, 7678 (1998)), who concluded that the third method is the most accurate (but it requires three additional FFTs).

If the same assignment function is used to distribute charges to the mesh and to assign forces to the charges, then Newton's third law is obeyed, and the total momentum of the system is conserved.